In [1]:
#r "nuget:System.Text.Json"
open System.Text.Json

Installing package System.Text.Json.done!

In [ ]:
//#r @"C:\Program Files (x86)\dotnet\shared\Microsoft.NETCore.App\3.1.0\System.Text.Json.dll"
#r @"C:\Users\arist\.nuget\packages\system.text.json\4.7.0\lib\net461\System.Text.Json.dll"

In [2]:
open System.Text.Json
open System.Text.Json.Serialization
open System
open System.IO
open System.Net.Http
open System.Net.Http.Headers

In [3]:
let BaseApiUrl = "https://www.kaggle.com/api/v1/"

type AuthorizedClient = AuthorizedClient of HttpClient

In [4]:
type Credentials =
    { Username: string
      Key: string }
    static member LoadFrom(path: string): Credentials =
        use reader = new StreamReader(path)
        let json = reader.ReadToEnd()
        JsonSerializer.Deserialize(json)

Unhandled Exception: input.fsx (1,1)-(1,1) interactive error Could not load file or assembly 'System.Text.Json, Version=4.0.1.0, Culture=neutral, PublicKeyToken=cc7b13ffcd2ddd51'.

In [12]:
let AuthoRizedClient(auth: Credentials) =
    let authToken =
        sprintf "%s:%s" auth.Username auth.Key
        |> Text.ASCIIEncoding.ASCII.GetBytes
        |> Convert.ToBase64String

    let client = new HttpClient()
    client.DefaultRequestHeaders.Authorization <- AuthenticationHeaderValue("Basic", authToken)

    AuthorizedClient client

Unhandled Exception: input.fsx (1,28)-(1,39) typecheck error The type 'Credentials' is not defined.
input.fsx (3,25)-(3,38) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (3,39)-(3,47) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.

In [13]:
let DownLoadFileAsync owner dataset filename destinationFolder (AuthorizedClient client) =
    let url = sprintf "%sdatasets/download/%s/%s/%s" BaseApiUrl owner dataset filename
    async {
        use! stream = client.GetStreamAsync(url) |> Async.AwaitTask
        use fstream = new FileStream(Path.Combine(destinationFolder, filename), FileMode.CreateNew)
        let! _ = stream.CopyToAsync fstream |> Async.AwaitTask
        fstream.Close()
        stream.Close()
    }